In [1]:
# 1. 필수 라이브러리 설치
!pip install -q transformers peft sacrebleu sentencepiece

# 2. 구글 드라이브 마운트 (학습된 모델을 불러오기 위해)
from google.colab import drive
drive.mount('/content/drive')

import torch
import gc
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import sacrebleu

# 3. 장치 설정 (Colab T4 GPU 사용 시 'cuda')
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Device: {device}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 3.2 MB/s eta 0:00:00
Mounted at /content/drive
✅ Device: cuda


In [2]:
OUTPUT_DIR = "/content/drive/MyDrive/V3_hyperclova-translator-mps"
MODEL_ID = "naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-0.5B"

# 1. 토크나이저 로드
print("📥 토크나이저 로딩 중...")
tokenizer = AutoTokenizer.from_pretrained(OUTPUT_DIR, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 2. Base Model 로드 (학습 안 된 원본)
print("📥 Base Model 로딩 중...")
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16,  # Colab T4 GPU에서는 float16 권장
    trust_remote_code=True,
    device_map="auto"
)

# 3. LoRA Adapter 결합 (학습된 가중치 합체)
print("🔗 LoRA Adapter 연결 중...")
try:
    model = PeftModel.from_pretrained(base_model, OUTPUT_DIR)
    model.eval()
    print("✅ 모델 로드 완료!")
except Exception as e:
    print(f"❌ 모델 로드 실패: {e}")
    print("경로(OUTPUT_DIR)가 정확한지 확인해주세요.")

📥 토크나이저 로딩 중...
📥 Base Model 로딩 중...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/1.13G [00:00<?, ?B/s]

🔗 LoRA Adapter 연결 중...
✅ 모델 로드 완료!


테스트1

In [3]:
# 생성 함수 정의
@torch.no_grad()
def generate(model, prompt: str, max_new_tokens=256, temperature=0.6) -> str:
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(device)

    out = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=(temperature > 0),
        temperature=temperature,
        repetition_penalty=1.2,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
    text = tokenizer.decode(out[0], skip_special_tokens=True)

    if "### Assistant:" in text:
        text = text.split("### Assistant:")[-1]
    return text.strip()

# 프롬프트 포맷 (학습 때와 동일해야 함)
SYSTEM_PROMPT = """너는 영미권 최신 슬랭/밈/관용 표현을 한국 인터넷 문화와 정서에 맞게 '초월 번역'하는 번역가다.

규칙:
1) 출력은 한국어 번역 1개만
2) 설명/해설/각주/분석 금지
3) 아래의 [참고] 내용은 번역 품질을 위한 힌트일 뿐이며, 출력에 절대 포함하지 말 것"""

def build_prompt(input_text, term="", meaning=""):
    ref = ""
    if term and meaning:
        ref = f"[참고]\n- 표현: {term}\n- 의미: {meaning}\n\n"

    user_part = (
        f"지시사항: Don't translate it in Korean, but translate it according to Korean culture\n\n"
        f"{ref}"
        f"입력 문장: {input_text}\n"
        f"출력 조건: 한국어 번역 1개만 출력"
    )

    return (
        "### System:\n"
        f"{SYSTEM_PROMPT}\n\n"
        "### User:\n"
        f"{user_part}\n\n"
        "### Assistant:\n"
    )

# 테스트 데이터 (직접 입력하거나 파일에서 로드)
test_data = [
    {
        "input": "She’s not just late, she runs on her own Disney time.",
        "term": "Disney time", "meaning": "늘 늦는 습관"
    },
    {
        "input": "Trying to get them to finish the report is like herding cats.",
        "term": "herding cats", "meaning": "통제 불능"
    },
    {
        "input": "He’s basically the office Santa Claus—shows up rarely but always hands out gifts.",
        "term": "", "meaning": ""
    },
    {
        "input": "He’s reading way too much into it—classic tinfoil-hat energy.",
        "term": "", "meaning": ""
    }
]

print("\n--- 🚀 테스트 시작 ---")
for item in test_data:
    prompt = build_prompt(item["input"], item["term"], item["meaning"])
    pred = generate(model, prompt)

    print(f"\n[Input]: {item['input']}")
    print(f"[Pred ]: {pred}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



--- 🚀 테스트 시작 ---

[Input]: She’s not just late, she runs on her own Disney time.
[Pred ]: 그녀만 늦게 가는 게 아니야. 혼자만의 '디즈니 시간'이 있어.

[Input]: Trying to get them to finish the report is like herding cats.
[Pred ]: 그들이 보고서 끝내도록 애쓰는 건 고양이 무리 이끌려는 것과 같어.

[Input]: He’s basically the office Santa Claus—shows up rarely but always hands out gifts.
[Pred ]: 걔는 그냥 사무실 산타 클라우스야. 별안간 와서 선물 빵빵 나눠주지.

[Input]: He’s reading way too much into it—classic tinfoil-hat energy.
[Pred ]: 걔 저 복잡한 거(맥락 분석) 좀 그만 읽어야 해. 완전 뚜렸뚜렷(흑역사 축적형 인간이야).


In [9]:
import pandas as pd
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer


# Base 모델 로드
base = AutoModelForCausalLM.from_pretrained(MODEL_ID, torch_dtype=torch.float16, trust_remote_code=True).to(device).eval()
# Tuned 모델 로드 (Base + LoRA Adapter)
tuned2 = PeftModel.from_pretrained(base, OUTPUT_DIR).to(device).eval()

# 2. 데이터 로드 및 전처리
file_path = "test_data.csv"
df = pd.read_csv(file_path)

# 컬럼명 매핑
test_df = df.rename(columns={
    "영어 원문 (Source Text)": "input",
    "초월 번역": "output"
})

# 테스트할 샘플 개수 선택 (랜덤 5개 또는 전체)
sampled_df = test_df.sample(50, random_state=42)

print(f"📊 테스트 데이터: 총 {len(sampled_df)}개 샘플링 (random state 고정)\n")

# 3. 프롬프트 및 생성 함수 설정
SYSTEM_PROMPT = """
You are a professional translator who specializes in translating English slang and memes into Korean internet slang and trendy expressions.
Your goal is to make the translation sound like a "close friend" or a "Korean netizen" speaking.

**CRITICAL RULES:**
1. **Never use polite language (Honorifics/Jon-dae-mal).** Use ONLY casual speech (Banmal).
2. Do not translate literally. Use Korean slang, memes, and community vibes aggressively.
3. If the original text is sarcastic or rude, preserve that tone perfectly.
4. Output ONLY the Korean translation. No explanations.
"""

def build_test_prompt(input_text):
    # 학습 데이터와 동일한 영어 Instruction 사용
    instruction = "Translate into Korean, but focus on cultural localization over literal meaning. Use trendy Korean internet slang and casual tone (Banmal) to match the vibe."

    user_content = (
        f"지시사항: {instruction}\n\n"
        f"입력 문장: {input_text}\n"
        f"출력 조건: 한국어 번역 1개만 출력"
    )

    return (
        "### System:\n" + SYSTEM_PROMPT.strip() + "\n\n"
        "### User:\n" + user_content + "\n\n"
        "### Assistant:\ncompletion: "
    )

@torch.no_grad()
def generate(model, prompt):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048).to(device)
    out = model.generate(
        **inputs,
        max_new_tokens=128,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.2,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )
    text = tokenizer.decode(out[0], skip_special_tokens=True)
    if "completion: " in text:
        return text.split("completion: ")[-1].strip()
    elif "### Assistant:" in text:
        return text.split("### Assistant:")[-1].strip()
    return text.strip()

# 4. 실행 및 결과 출력 (요청하신 포맷 적용)
print(f"{'='*20} [테스트 결과 비교] {'='*20}\n")

for idx, row in sampled_df.iterrows():
    input_text = row['input']
    ref_text = row['output']

    prompt = build_test_prompt(input_text)

    # 모델 생성
    base_out = generate(base, prompt)
    tuned_out = generate(tuned2, prompt)

    print(f"[Sample {idx}]")
    print(f"원문            : {input_text}")
    print(f"초월번역(정답데이터): {ref_text}")
    print(f"base model     : {base_out}")
    print(f"Tuned model    : {tuned_out}")
    print("-" * 60)

📊 테스트 데이터: 총 50개 샘플링 (random state 고정)

==================== [테스트 결과 비교] ====================

[Sample 62]
원문            : There is a tide in the affairs of men... leads on to fortune.
초월번역(정답데이터): 물들어 올때 노저어야 한다. 
base model     : 남사탕이 돌아오면 세상도 편해지겠어.
Tuned model    : 남사스러운 일이 일어나면 결국 운도 따라붙기 마련이지.
------------------------------------------------------------
[Sample 40]
원문            : They expected him to crack under pressure, but he stayed cool as a cucumber.
초월번역(정답데이터): 다 긴장했는데 걔 혼자 태평하더라
base model     : 그들은 그에게 압살할 거라 예상했지만, 그는 스트레스 받을 때 Like a intruder(도둑) 굴러들어오는 줄 몰랐어.
Tuned model    : 그는 압력을 받으면 깨질 거라고 예상했지만, 잘 버텨냈다(견뎌냈다).
------------------------------------------------------------
[Sample 95]
원문            : He's been a couch potato all weekend.
초월번역(정답데이터): 그 남자는 주말 내내 소파에서 움직이질 않아.
base model     : 주말 내내 게임만 하네(완전 게임이네).
Tuned model    : 주말 내내 게임만 하더라(완전 게임이네).
------------------------------------------------------------
[Sample 18]
원문            : Given the complexity of